# 2D Heat Diffusion using Finite Difference Method

Similarly to the 1D case, the 2D heat equation is given by:

$$\frac{\partial u}{\partial t} = \alpha (\frac{\partial^2 u}{\partial x^2} + \frac{\partial^2 u}{\partial y^2})$$

Discretized in position and time the equation is:
$$\frac{u_{new} - u_{old}}{\Delta t} = \alpha (\frac{u_{old\_left} - 2u_{old} + u_{old\_right}}{\Delta x^2} + \frac{u_{old\_down} - 2u_{old} + u_{old\_up}}{\Delta x^2})$$

To improve stability, we instead use the implicit method, which is given by:

$$\frac{u_{new} - u_{old}}{\Delta t} = \alpha (\frac{u_{new\_left} - 2u_{new} + u_{new\_right}}{\Delta x^2} + \frac{u_{new\_down} - 2u_{new} + u_{new\_up}}{\Delta x^2})$$


$$u_{old} = u_{new} - \alpha \Delta t (\frac{u_{new\_left} - 2u_{new} + u_{new\_right}}{\Delta x^2} + \frac{u_{new\_down} - 2u_{new} + u_{new\_up}}{\Delta x^2})$$

let $r = \alpha \Delta t / \Delta x^2$

$$u_{old} = u_{new} - r u_{new\_left} + 2r u_{new} - r u_{new\_right} - r u_{new\_down} + 2r u_{new} - r u_{new\_up}  $$
$$u_{old} = (1 + 4r) u_{new} - r u_{new\_left} - r u_{new\_right} - r u_{new\_down} - r u_{new\_up}  $$

In [ ]:
# Start with inital conditions
import numpy as np
from matplotlib import pyplot as plt

GRID_SIZE = 50


def initalize_checkerboard(temps, checker_size=10):
    for i in range(0, GRID_SIZE, checker_size):
        for j in range(0, GRID_SIZE, checker_size):
            if (i // checker_size + j // checker_size) % 2 == 0:
                temps[i:i+checker_size, j:j+checker_size] = 1.0

temps = np.zeros((GRID_SIZE, GRID_SIZE))
initalize_checkerboard(temps)
plt.imshow(temps, cmap='hot', interpolation='nearest', origin='lower')

In [2]:
boundary_mask = np.zeros((GRID_SIZE, GRID_SIZE), dtype=bool)
boundary_mask[0, :] = True
boundary_mask[-1, :] = True
boundary_mask[:, 0] = True
boundary_mask[:, -1] = True

# Set boundary conditions to be the same as the initial conditions for boundary cells
boundary_conditions = temps.copy()

In [3]:
def get_id_from_cell(i, j):
    return i * GRID_SIZE + j

def get_cell_from_id(id):
    return id // GRID_SIZE, id % GRID_SIZE

In [ ]:
from tqdm import tqdm
from time import time

DELTA_T = 0.1
DELTA_X = 1.0
CONDUCTIVITY = 0.01

TIME_STEPS = 100

# Interestingly, the matrix A is the same for every time step and will be the same of node sizes/conductivity or the time step doesn't change
def compute_A():
    r = CONDUCTIVITY * DELTA_T / DELTA_X**2
    A = np.zeros((GRID_SIZE**2, GRID_SIZE**2))
    for i in range(GRID_SIZE):
        for j in range(GRID_SIZE):
            if boundary_mask[i][j] == True:
                id = get_id_from_cell(i, j)
                A[id, id] = 1
            else:
                id = get_id_from_cell(i, j)
                A[id, id] = 1 + 4 * r
                A[id, get_id_from_cell(i-1, j)] = -r
                A[id, get_id_from_cell(i+1, j)] = -r
                A[id, get_id_from_cell(i, j-1)] = -r
                A[id, get_id_from_cell(i, j+1)] = -r

    return A

def step_slow(temps, A):
    # Create a matrix to represent the system of equations
    b = np.zeros(GRID_SIZE**2)

    for i in range(GRID_SIZE):
        for j in range(GRID_SIZE):
            if boundary_mask[i][j] == True:
                id = get_id_from_cell(i, j)
                b[id] = boundary_conditions[i, j]

            else:
                id = get_id_from_cell(i, j)
                b[id] = temps[i, j]

    # This is the slow part!
    new_temps = np.linalg.solve(A, b).reshape(GRID_SIZE, GRID_SIZE)
    return new_temps

temps = np.zeros((GRID_SIZE, GRID_SIZE))
initalize_checkerboard(temps)
A = compute_A()

start_time = time()
for i in tqdm(range(TIME_STEPS)):
    temps = step_slow(temps, A)
print("Time taken: ", time() - start_time)

ground_truth = temps.copy()

plt.imshow(temps, cmap='hot', interpolation='nearest', origin='lower')

# Too slow
Using a traditional solver, it takes too long. Instead, we will use an iterative solver to solver our sparse matrix.

SciPy has many options so we will figure out which one is best for our problem.

In [ ]:
from tqdm import tqdm
from time import time
from scipy.sparse.linalg import bicg, bicgstab, cg, cgs, gmres, lgmres, minres, qmr, gcrotmk, tfqmr

solve_time = 0

def step(temps, A, solver=cg):
    global solve_time
    # Create a matrix to represent the system of equations
    b = np.zeros(GRID_SIZE**2)

    for i in range(GRID_SIZE):
        for j in range(GRID_SIZE):
            if boundary_mask[i][j] == True:
                id = get_id_from_cell(i, j)
                b[id] = boundary_conditions[i, j]

            else:
                id = get_id_from_cell(i, j)
                b[id] = temps[i, j]

    # Use a sparse solver
    start_time = time()
    new_temps = solver(A, b)[0].reshape(GRID_SIZE, GRID_SIZE)
    solve_time = time() - start_time

    return new_temps

solvers = [cg, bicg, bicgstab, cgs, gmres, lgmres, minres, qmr, gcrotmk, tfqmr]
data = []

for solver in solvers:
    temps = np.zeros((GRID_SIZE, GRID_SIZE))
    initalize_checkerboard(temps)
    A = compute_A()

    solve_times = []
    for i in tqdm(range(TIME_STEPS)):
        temps = step(temps, A, solver)
        solve_times.append(solve_time)

    data.append({
        "Solver": solver.__name__,
        "Time": sum(solve_times) / len(solve_times),
        "Error": np.linalg.norm(ground_truth - temps)
    })

In [ ]:
import pandas as pd
df = pd.DataFrame(data)
plt.scatter(df["Time"], df["Error"], label=df["Solver"])
plt.xlabel("Time taken per iteration")
plt.ylabel("Error")
for i, row in df.iterrows():
    plt.text(row["Time"], row["Error"], row["Solver"])
df

In [53]:
BEST_SOLVER = bicgstab

In [ ]:
# Create animation
SAVE_FREQUENCY = 50
historical_temps = []

temps = np.zeros((GRID_SIZE, GRID_SIZE))
initalize_checkerboard(temps)

A = compute_A()

for i in tqdm(range(20_000)):
    if i % SAVE_FREQUENCY == 0:
        historical_temps.append(temps.copy())
    temps = step(temps, A, BEST_SOLVER)

plt.imshow(temps, cmap='hot', interpolation='nearest', origin='lower')

In [ ]:
# Create an animation
import os
os.makedirs("output", exist_ok=True)

for i in tqdm(range(len(historical_temps))):
    plt.figure(figsize=(10, 10))
    plt.imshow(historical_temps[i], cmap='gray', interpolation='nearest', origin='lower')
    plt.title(f"Step {i * SAVE_FREQUENCY}")
    plt.tight_layout()
    plt.savefig(f"output/{i:04d}.png")
    plt.close()

In [ ]:
# Use ffmpeg to create a video. Start with making the intro using only the first frame for 3 seconds
!ffmpeg -y -framerate 0.3 -i output/0000.png -c:v libx264 -r 30 -pix_fmt yuv420p output/intro.mp4
# Then create the video
!ffmpeg -y -framerate 30 -i output/%04d.png -c:v libx264 -r 30 -pix_fmt yuv420p output/animation.mp4
# Concatenate the intro and the video
!ffmpeg -y -i output/intro.mp4 -i output/animation.mp4 -filter_complex "[0:v] [1:v] concat=n=2:v=1 [v]" -map "[v]" output/output.mp4